# Equal-Weight S&P 500 Index Fund

##  Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell us how many shares of each S&P 500 constituent we should purchase to get an equal-weight version of the index fund.


In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

list of stock


In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [19]:
##REQUIRES VALID API to function. My free trial has expired

IEX_CLOUD_API_TOKEN = 'sk_84426c6aa28347098505be7f9d3378bb'


## First API Call


* Market capitalization for each stock
* Price of each stock



In [20]:
symbol = 'AAPL'
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}"
print(api_url)

response = requests.get(api_url)

if response.status_code == 200:
    try:
        data = response.json()
        print(data)
    except ValueError:
        print("Response is not in valid JSON format:", response.content)
else:
    print("Error:", response.status_code, response.reason)

https://sandbox.iexapis.com/stable/stock/AAPL/quote?token=sk_84426c6aa28347098505be7f9d3378bb
Error: 403 Production tokens cannot be used in the sandbox environment. Please use https://cloud.iexapis.com


## Parsing  API Call


In [9]:
price = data['latestPrice']
market_cap = data['marketCap']


NameError: name 'data' is not defined

## Adding  Stocks Data to a Pandas DataFrame



In [10]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns=my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [11]:
final_dataframe.append(
    pd.Series(
        [
            symbol,
            price,
            market_cap,
            'N/A'
        ]
        ,
    index = my_columns
    )
    ,
    ignore_index=True
)

NameError: name 'price' is not defined

## Looping Through The Tickers in  List of Stocks


In [12]:

for stock in stocks['Ticker']:
    api_url = f"https://sandbox.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
    pd.Series(
    [
        stock, 
        data['latestPrice'],
        data['marketCap'],
        'N/A'
    ]
        ,
    index = my_columns
    )
    ,
    ignore_index=True
)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
final_dataframe

## Using Batch API Calls to Improve Performance



In [13]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [14]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
 #   print(symbol_strings)
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol == 'WLTW':
            continue
        price = data[symbol]['quote']['latestPrice']
        capit = data[symbol]['quote']['marketCap']
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            price, 
            capit,
            'N/A'
        ], index = my_columns),
        ignore_index=True
        )
print(final_dataframe)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
#print(final_dataframe)

## Calculating the Number of Shares to Buy



In [14]:
portfolio_size = input('Enter the value of your portfolio: ')
try:
    val = float(portfolio_size)
except ValueError:
    print('That is not a number')
    portfolio_size = input('Enter the value of your portfolio: ')
    val = float(portfolio_size)

Enter the value of your portfolio: 1000000


In [15]:
portfolio_size

'1000000'

In [16]:
position_size = val/ len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, "Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[i, "Stock Price"])
final_dataframe.head()

#number_apple_share = position_size/500
#print(math.floor(number_apple_share))

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,140.10,42597817186,14
1,AAL,16.50,10896349403,120
2,AAP,235.94,14511385956,8
3,AAPL,163.73,2680866374359,12
4,ABBV,137.16,242671071016,14


## Formatting  Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.


In [32]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, 'Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [33]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
{
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
    
}
)


dollar_format = writer.book.add_format(
{
    'num_format': '$0.00',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
    
}
)


integer_format = writer.book.add_format(
{
    'num_format': '0',
    'font_color': font_color,
    'bg_color': background_color,
    'border': 1
    
}
)


### Applying the Formats to the Columns of  `.xlsx` File


```

In [34]:
writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)


writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
writer.sheets['Recommended Trades'].write('B1', 'Stock Price', string_format)
writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
writer.sheets['Recommended Trades'].write('D1', 'Number of Shares to Buy', string_format)

0

## Saving Our Excel Output

Saving our Excel file is very easy:

In [35]:
writer.save()